 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Utiliser NLPModels pour l'optimisation sans contraintes
Tangi Migot

In this tutorial, we introduce NLPModels for numerical optimization. In particular, we will see how to create a structure providing the derivatives of an optimization problem, and then how we can use Ipopt to solve it.

In this short tutorial, we will use three new packages that belongs to the organization [JuliaSmoothOptimizers](https://juliasmoothoptimizers.github.io/) developed at Polytechnique Montréal.

In [ ]:
using Pkg; 
Pkg.activate("nlpmodels") # use the closed environment defined in the nlpmodels folder

In [ ]:
using ADNLPModels #Pkg.add("ADNLPModels")
using NLPModels #Pkg.add("NLPModels")
using NLPModelsJuMP #Pkg.add("NLPModelsJuMP")
using NLPModelsIpopt #Pkg.add("NLPModelsIpopt")

There exists an online documentation for them:
- NLPModels: [https://juliasmoothoptimizers.github.io/NLPModels.jl/latest/tutorial/](https://juliasmoothoptimizers.github.io/NLPModels.jl/latest/tutorial/)
- NLPModelsJuMP: [https://juliasmoothoptimizers.github.io/NLPModelsJuMP.jl/dev/tutorial/](https://juliasmoothoptimizers.github.io/NLPModelsJuMP.jl/dev/tutorial/)
- NLPModelsIpopt: [https://juliasmoothoptimizers.github.io/NLPModelsIpopt.jl/stable/tutorial/#Tutorial-1](https://juliasmoothoptimizers.github.io/NLPModelsIpopt.jl/stable/tutorial/#Tutorial-1)
- ADNLPModels: [https://juliasmoothoptimizers.github.io/ADNLPModels.jl/latest/tutorial/](https://juliasmoothoptimizers.github.io/ADNLPModels.jl/latest/tutorial/)

## NLPModels

What is NLPModels? An NLPModel is a structure allowing us to easily access the derivatives of an optimization problem. There are two principle ways to create such a structure:
i)  Provide the objective function to ADNLPModel, and uses Julia's automatic differentiation;
ii) Convert a problem created in JuMP to an NLPModel.

### 1) Using automatic differentiation

For this purpose, NLPModels has a structure called *ADNLPModel* which is used for unconstrained optimization as follows.

In [ ]:
using ADNLPModels #call the package ADNLPModels

#Initialize the objective function, and an initial guess
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2
x0 = [-1.2; 1.0]

#Create an ADNLPModel
nlp = ADNLPModel(f, x0)

In the print, we can already see a number of important information:
- *ADNLPModel - Model with automatic differentiation* as planned Julia uses the automatic differentiation.
- There are a number information relative to the size of the problem. These information are stored in the **meta**.
- There are **Counters**, which are keeping track of the number of evaluations of the objective function, gradient, hessian ...

In [ ]:
print(nlp.meta)

In [ ]:
nlp.meta.nvar #returns the size of x

In [ ]:
print(nlp.counters)

In [ ]:
neval_obj(nlp) #get the number of evaluation of the objective function

To evaluate the objective function and its derivatives, we have at hand the following functions:
- `obj`
- `grad`
- `hess`
- `hprod`
- and more...

In [ ]:
x = ones(2)
obj(nlp, x)
grad(nlp, x)
v = rand(2) #vector of size 2 with random numbers between 0 and 1
hprod(nlp, x, v)

It is very important to note here that the function `hess` returns only the lower triangular of the hessian matrix, since it is always a symmetric matrix and it uses less memory this way.

In [ ]:
hess(nlp, x)

In [ ]:
using LinearAlgebra

In [ ]:
hprod(nlp, x, v) - Symmetric(hess(nlp, x), :L) * v

### 2) Use a JuMP model

For this purpose, we use a package associated to `NLPModels`, which is called `NLPModelsJuMP`.

In [ ]:
using NLPModels, NLPModelsJuMP, JuMP

x0 = [-1.2; 1.0]
model = Model() # No solver is required
@variable(model, x[i=1:2], start=x0[i])
@NLobjective(model, Min, (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2)

nlp_jump = MathOptNLPModel(model)

To access the objective function and the derivatives, we proceed the exact same way as for `ADNLPModel` before.

In [ ]:
x = ones(2)
obj(nlp_jump, x)
grad(nlp_jump, x)
v = rand(2)
hprod(nlp_jump, x, v)
hess(nlp_jump, x) #sparse matrix

## Solve an NLPModel with Ipopt

The problem created with `JuMP` can be solved with `Ipopt`.

In [ ]:
using JuMP, Ipopt

model = Model(Ipopt.Optimizer)
x0 = [-1.2; 1.0]
@variable(model, x[i=1:2], start=x0[i])
@NLobjective(model, Min, (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2)
optimize!(model)

The same can be achieved very easily with `NLPModels` using the package `NLPModelsIpopt` and its function `ipopt`.

In [ ]:
using NLPModels, NLPModelsIpopt

nlp = ADNLPModel(x -> (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2, [-1.2; 1.0])
stats = ipopt(nlp)
print(stats)